In [1]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np
from tensorflow.keras.optimizers import Adam

In [2]:
model = load_model("./model_mobileNetv2_1.h5")
model.compile(optimizer = Adam(learning_rate=1e-4), #指定學習率
            loss = 'categorical_crossentropy',
            metrics = ['acc']
            )

In [ ]:
model.summary()

In [3]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +"haarcascade_frontalface_alt2.xml")
video_capture = cv2.VideoCapture(0)
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    faces_list = []
    preds = []
    for (x, y, w, h) in faces:
        face_frame = frame[y:y+h, x:x+w]
        face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
        face_frame = cv2.resize(face_frame, (224, 224))
        face_frame = img_to_array(face_frame)
        face_frame = np.expand_dims(face_frame, axis=0)
        face_frame = preprocess_input(face_frame)
        faces_list.append(face_frame)
    
    if len(faces_list) > 0:
        preds = model.predict(faces_list)
    else:
    # 當未偵測到人臉時的處理邏輯
        print("未偵測到人臉")
        continue  # 跳過本次迴圈，繼續下一個幀的處理
    
    for pred in preds:
        max_index = np.argmax(pred)
        if max_index == 0:
            label = "Incorrect Mask"
            color = (0, 165, 255)  # Orange color for incorrect mask
        elif max_index == 1:
            label = "No Mask"
            color = (0, 0, 255)  # Red color for no mask
        else:
            label = "Mask"
            color = (0, 255, 0)  # Green color for correct mask

        label = "{}: {:.2f}%".format(label, pred[max_index] * 100)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 52ms/step
